In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report   



In [21]:
df = pd.read_csv('medicine_recommendation_dataset.csv')
df.head()

,Symptom,Disease,Recommended Medicine,Category,Dosage Form,Strength (mg),Manufacturer,Prescription Type,Nearest Pharmacy,City
0,Dizziness,Low BP,Midodrine,Vasoconstrictor,Tablet,730.0,Eli Lilly,Over-the-Counter,CareWell Pharmacy,Islamabad
1,Shortness of Breath,Asthma,Salbutamol,Bronchodilator,Capsule,936.0,NaN,Over-the-Counter,TrustPharma,Faisalabad
2,Eye Infection,Conjunctivitis,Tobramycin,Antibacterial,Cream,714.0,Bayer,Prescription,WellLife,Lahore
3,Skin Rash,Fungal Infection,Clotrimazole,Antifungal,Capsule,598.0,Teva,Prescription,TrustPharma,Multan
4,"Fever, Headache",Viral Infection,Paracetamol,Antipyretic,Syrup,190.0,Pfizer,Prescription,CareWell Pharmacy,Islamabad


In [22]:
for i in df.columns:
    print(i, ":", df[i].nunique())
    print(df[i].value_counts())

Symptom : 20
Symptom
Skin Rash              28
Sore Throat            28
Dizziness              23
Ear Pain               20
Vomiting, Nausea       20
Toothache              20
Fever, Headache        19
Abdominal Pain         18
Shortness of Breath    18
Back Pain              18
Eye Infection          17
High Sugar, Fatigue    16
Cough, Chest Pain      16
Cold, Sneezing         15
Urinary Pain           14
Migraine               14
Anxiety, Insomnia      13
Burn Injury            12
Body Pain              11
Joint Pain              8
Name: count, dtype: int64
Disease : 20
Disease
Fungal Infection    28
Infection           28
Low BP              23
Otitis              21
Dental Infection    20
Food Poisoning      20
Viral Infection     19
Ulcer               18
Asthma              18
Spinal Pain         18
Conjunctivitis      17
Diabetes            16
Bronchitis          15
Headache            15
Common Cold         15
UTI                 14
Depression          13
Muscle Pain         1

In [23]:
df.duplicated().sum()

0

In [24]:
df.isnull().sum()

Symptom                 2
Disease                 3
Recommended Medicine    0
Category                0
Dosage Form             0
Strength (mg)           4
Manufacturer            2
Prescription Type       0
Nearest Pharmacy        0
City                    0
dtype: int64

In [25]:
df['Symptom'].value_counts()

Symptom
Skin Rash              28
Sore Throat            28
Dizziness              23
Ear Pain               20
Vomiting, Nausea       20
Toothache              20
Fever, Headache        19
Abdominal Pain         18
Shortness of Breath    18
Back Pain              18
Eye Infection          17
High Sugar, Fatigue    16
Cough, Chest Pain      16
Cold, Sneezing         15
Urinary Pain           14
Migraine               14
Anxiety, Insomnia      13
Burn Injury            12
Body Pain              11
Joint Pain              8
Name: count, dtype: int64

In [26]:
df1 = pd.read_csv('medicine_recommendation_dataset.csv')
df1

,Symptom,Disease,Recommended Medicine,Category,Dosage Form,Strength (mg),Manufacturer,Prescription Type,Nearest Pharmacy,City
0,Dizziness,Low BP,Midodrine,Vasoconstrictor,Tablet,730.0,Eli Lilly,Over-the-Counter,CareWell Pharmacy,Islamabad
1,Shortness of Breath,Asthma,Salbutamol,Bronchodilator,Capsule,936.0,NaN,Over-the-Counter,TrustPharma,Faisalabad
2,Eye Infection,Conjunctivitis,Tobramycin,Antibacterial,Cream,714.0,Bayer,Prescription,WellLife,Lahore
3,Skin Rash,Fungal Infection,Clotrimazole,Antifungal,Capsule,598.0,Teva,Prescription,TrustPharma,Multan
4,"Fever, Headache",Viral Infection,Paracetamol,Antipyretic,Syrup,190.0,Pfizer,Prescription,CareWell Pharmacy,Islamabad
...,...,...,...,...,...,...,...,...,...,...
345,"Cough, Chest Pain",Bronchitis,Azithromycin,Antibiotic,Ointment,275.0,Sanofi,Over-the-Counter,MediCare,Islamabad
346,Back Pain,Spinal Pain,Naproxen,Pain Reliever,Ointment,208.0,Sanofi,Prescription,Sehat Pharmacy,Islamabad
347,"Vomiting, Nausea",Food Poisoning,Ondansetron,Antiemetic,Drops,717.0,Merck,Prescription,DawaCenter,Islamabad
348,"Cold, Sneezing",Common Cold,Cetirizine,Antihistamine,Inhaler,630.0,Sanofi,Prescription,DawaCenter,Faisalabad


In [27]:
df1.isnull().sum()

Symptom                 2
Disease                 3
Recommended Medicine    0
Category                0
Dosage Form             0
Strength (mg)           4
Manufacturer            2
Prescription Type       0
Nearest Pharmacy        0
City                    0
dtype: int64

In [28]:
df1.dropna(inplace=True)

In [29]:
def filling(data):
    def filling(data=None, cols=None, inplace=True):
        """
        Fill missing values by column mode.
        - data can be: None (use df if available else df1), a DataFrame, or a column-name string.
        - cols: optional list of columns to fill (only existing columns with missing values will be used).
        - inplace: if True, fills the passed/selected DataFrame in place and returns it.
        Prints missing-value counts for columns being filled and returns the filled DataFrame.
        """
        # resolve target dataframe
        if isinstance(data, str):
            col = data
            if 'df1' in globals() and col in df1.columns:
                target = df1
            elif 'df' in globals() and col in df.columns:
                target = df
            else:
                raise ValueError(f"Column {col!r} not found in df1 or df")
            cols = [col]
        elif data is None:
            if 'df' in globals():
                target = df
            elif 'df1' in globals():
                target = df1
            else:
                raise ValueError("No target DataFrame found (neither df nor df1 present).")
        elif hasattr(data, 'columns'):
            target = data
        else:
            raise TypeError("Expected a DataFrame, a column-name string, or None.")

        # decide which columns to fill
        if cols is None:
            cols_to_fill = [c for c in target.columns if target[c].isnull().sum() > 0]
        else:
            cols_to_fill = [c for c in cols if c in target.columns and target[c].isnull().sum() > 0]

        if not cols_to_fill:
            print("No missing values found for the requested columns.")
            return target

        print("Missing values by column:")
        for c in cols_to_fill:
            print(f"{c:22} {target[c].isnull().sum()}")

        for c in cols_to_fill:
            mode = target[c].mode(dropna=True)
            if mode.empty:
                print(f"Skipping {c!r}: no mode available")
                continue
            fill_val = mode.iloc[0]
            target[c].fillna(fill_val, inplace=inplace)
            print(f"Filled {c!r} with mode: {fill_val!r}")

        return target

    # Example: fill missing columns in original df (only those with missing values will be touched)
    if 'df' in globals():
        df = filling(df)

In [30]:
df1['Symptom'].value_counts()

Symptom
Sore Throat            28
Skin Rash              26
Dizziness              23
Toothache              20
Vomiting, Nausea       20
Ear Pain               20
Back Pain              18
Fever, Headache        18
Abdominal Pain         17
Eye Infection          17
Shortness of Breath    17
High Sugar, Fatigue    16
Cold, Sneezing         15
Cough, Chest Pain      15
Urinary Pain           14
Migraine               14
Anxiety, Insomnia      13
Body Pain              11
Burn Injury            11
Joint Pain              6
Name: count, dtype: int64

In [31]:
import re

# replace common two-symptom patterns robustly (handles optional spaces and both orders)
df1['Symptom'] = df1['Symptom'].str.replace(r'\bVomiting\s*,\s*Nausea\b', 'Nausea', regex=True)
df1['Symptom'] = df1['Symptom'].str.replace(r'\bNausea\s*,\s*Vomiting\b', 'Nausea', regex=True)
df1['Symptom'] = df1['Symptom'].str.replace(r'\bHeadache\s*,\s*Dizziness\b', 'Dizziness', regex=True)
df1['Symptom'] = df1['Symptom'].str.replace(r'\bDizziness\s*,\s*Headache\b', 'Dizziness', regex=True)

# normalize surrounding whitespace first
df1['Symptom'] = df1['Symptom'].str.strip()

# handle common two-symptom combinations in either order (case-insensitive, optional spaces around comma)
pairs = [
    (('Abdominal Pain', 'Nausea'), 'Abdominal Pain'),
    (('Anxiety', 'Insomnia'), 'Anxiety'),
    (('Cold', 'Sneezing'), 'Cold'),
    (('Cough', 'Chest Pain'), 'Chest Pain'),
    (('High Sugar', 'Fatigue'), 'Fatigue'),
    (('Back Pain', 'Muscle Ache'), 'Muscle Ache'),
    (('Fever', 'Headache'), 'Fever'),
    (('Constipation', 'Abdominal Pain'), 'Abdominal Pain'),
]

for (a, b), repl in pairs:
    pattern = rf'(?i)\b(?:{re.escape(a)}\s*,\s*{re.escape(b)}|{re.escape(b)}\s*,\s*{re.escape(a)})\b'
    df1['Symptom'] = df1['Symptom'].str.replace(pattern, repl, regex=True)

# final cleanup
df1['Symptom'] = df1['Symptom'].str.strip()




In [32]:
df1['Symptom'].value_counts()

Symptom
Sore Throat            28
Skin Rash              26
Dizziness              23
Toothache              20
Nausea                 20
Ear Pain               20
Back Pain              18
Fever                  18
Abdominal Pain         17
Eye Infection          17
Shortness of Breath    17
Fatigue                16
Cold                   15
Chest Pain             15
Urinary Pain           14
Migraine               14
Anxiety                13
Body Pain              11
Burn Injury            11
Joint Pain              6
Name: count, dtype: int64

In [33]:
df1.drop('Dosage Form', axis=1, inplace=True)
df1.drop('Category', axis=1, inplace=True)


In [34]:
def encoding(data):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for i in data.columns:
        data[i] = le.fit_transform(data[i])
    return data
encoding(df1)


,Symptom,Disease,Recommended Medicine,Strength (mg),Manufacturer,Prescription Type,Nearest Pharmacy,City
0,7,13,10,193,3,0,0,1
2,9,5,19,187,2,1,9,3
3,16,10,5,157,12,1,8,4
4,11,19,15,23,8,1,0,1
5,0,18,13,233,3,1,4,1
...,...,...,...,...,...,...,...,...
345,5,2,2,58,10,0,6,1
346,2,16,11,30,10,1,7,1
347,14,9,14,188,6,1,3,1
348,6,4,3,164,10,1,3,0


In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 0 to 349
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Symptom               339 non-null    int32
 1   Disease               339 non-null    int32
 2   Recommended Medicine  339 non-null    int32
 3   Strength (mg)         339 non-null    int64
 4   Manufacturer          339 non-null    int32
 5   Prescription Type     339 non-null    int32
 6   Nearest Pharmacy      339 non-null    int32
 7   City                  339 non-null    int32
dtypes: int32(7), int64(1)
memory usage: 14.6 KB


In [36]:
X = df1.drop('Recommended Medicine', axis=1)
y = df1['Recommended Medicine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((271, 7), (68, 7), (271,), (68,))

In [37]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         2
           2       0.67      1.00      0.80         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      0.80      0.89         5
          10       1.00      1.00      1.00         7
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       0.89      1.00      0.94         8
          15       1.00      1.00      1.00         3
          16       1.00      1.00      1.00         2
          17       1.00    

In [38]:
# import pickle as pkl
# with open('medicine_model.pkl', 'wb') as f:
#     pkl.dump(model, f)



In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load dataset
df1 = pd.read_csv("medicine_recommendation_dataset.csv")

# Select required columns
df1 = df1[['Symptom', 'Disease','Recommended Medicine']]


# Create encoders
encoders = {}
for col in df1.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# Split data
X = df[['Symptom', 'Disease']]
y = df['Recommended Medicine']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
with open("medicine_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save encoders
with open("encoders.pkl", "wb") as f:
    pickle.dump(encoders, f)

print("✅ Model and encoders saved successfully")


✅ Model and encoders saved successfully
